### Load packages

In [1]:
import os, sys
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
import zarr
import napari

### Start Napari viewer

In [2]:
%gui qt
napari.gui_qt()
viewer = napari.Viewer(ndisplay=3)

C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\napari_tools_menu\__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


### Specify directory to data files

In [3]:
roi_dir='test-oct6_epsilon1\\roi.csv'
lb_dir='lt172_gene_r3\\segmentation\\lt172_gene_r3-c3.tif'
spot_dir='D:\\SWAP\\Vincent\\lt172_exps\\gene\\outputs\\spots_pooled\\spots_r3_c0.txt'
imdir="lt172_gene_r3\\export.n5"

### Specify ROI ID, block size and image channel 

In [4]:
#Input information here: cell ID, image block size, channels
cell_id = 15 

#define the size of the block to display around the ROI
dx,dy,dz=[100,100,50]  # in pixel 
channel_1='c3'
channel_2='c0'

### Load data

In [5]:
roi = pd.read_csv(roi_dir,sep=',', index_col=0) #, index_col=1)
roi

,z,y,x,area
roi,,,,
1,158.002679,113.900479,250.167418,1469.498520
2,295.632291,50.414473,255.207682,1284.644760
3,158.609342,43.333067,347.254401,1356.453336
4,281.253706,57.066587,306.891305,1508.424456
5,279.531618,156.364789,237.923032,1414.131264
...,...,...,...,...
10038,185.404246,135.635883,183.955516,164.502072
10039,91.481847,46.423266,134.057193,468.533184
10040,163.406428,164.625958,75.953493,223.157592


In [6]:
# get n5 image data for channel 1
im = zarr.open(store=zarr.N5Store(imdir), mode='r')
_img = im[channel_1+'/s2']
zdim, ydim, xdim = _img.shape

In [7]:
lb=imread(lb_dir)
z=roi.loc[cell_id,'z']*2/0.84 # convert to pixel 
y=roi.loc[cell_id,'y']*2/0.92 # convert to pixel 
x=roi.loc[cell_id,'x']*2/0.92 # convert to pixel 
Z=int(z)
Y=int(y)
X=int(x)

z_min, z_max = max(0, Z-dz), min(zdim, Z+dz)
y_min, y_max = max(0, Y-dy), min(ydim, Y+dy)
x_min, x_max = max(0, X-dx), min(xdim, X+dx)

#Crop image to show the target cell and surrounding area
lb_sub=lb[z_min:z_max, y_min:y_max, x_min:x_max]
#Target Cell ID
z0 = z - z_min
y0 = y - y_min
x0 = x - x_min
a = np.array([[z0, y0, x0]])
a

array([[ 50.56512472, 100.1071203 , 100.51405115]])

### Display segmentation mask, image data and the position of the specified ROI

In [8]:
# get n5 image data for channel 1
im = zarr.open(store=zarr.N5Store(imdir), mode='r')
img_1 = im[channel_1+'/s2'][z_min:z_max, y_min:y_max, x_min:x_max]
viewer.add_image(img_1,colormap='green',blending='additive')

#get n5 image data for channel 2
if channel_2!='':
    im = zarr.open(store=zarr.N5Store(imdir), mode='r')   
    img_2 = im[channel_2+'/s2'][z_min:z_max, y_min:y_max, x_min:x_max]
    viewer.add_image(img_2.astype('float32'),colormap='gray',blending='additive')
    
# x, y, z are cell centroid location (in pixel)
zo = z - z_min
yo = y - y_min
xo = x - x_min

a = np.array([[zo, yo, xo]])

viewer.add_points(a, size=5, face_color='red',edge_color='red',name='cell_centroid_position')
viewer.add_labels(lb_sub.astype('uint32'),name='labels',blending='additive')

C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


<Labels layer 'labels' at 0x23449bfb640>

### Spot detection can also be visualized 

In [9]:
spot=np.loadtxt(spot_dir,delimiter=',')
vox=[0.92,0.92,0.84]  #current image voxel size
spot[:,:3]=spot[:,:3]/vox

### Display spot detection in selected image block

In [10]:
_x = spot[:,0]
_y = spot[:,1]
_z = spot[:,2]
conditions = [
    _x <= x_max,
    _y <= y_max,
    _z <= z_max,
    
    _x >  x_min,
    _y >  y_min,
    _z >  z_min,
]
cond = (np.sum(conditions, axis=0) == 6) # all six conditions has to be satified to be pulled

In [11]:
b = spot[cond]

bx=b[:,0]-x_min
by=b[:,1]-y_min
bz=b[:,2]-z_min
point=np.stack((bz,by,bx),axis=-1)
viewer.add_points(point, size=1, face_color='red',edge_color='red',name='detected_spots')

C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


<Points layer 'detected_spots' at 0x2344985bca0>

In [12]:
f = 'D:\\SWAP\\Vincent\\lt172_exps\\gene\\outputs\\test-oct6_epsilon1\\clustering.csv'
res = pd.read_csv(f, index_col=0)
res

,r1_c0,r1_c1,r1_c2,r1_c4,r3_c0,r3_c1,r3_c2,r3_c4,r4_c0,r4_c1,...,r5_c4_log2,ybin,xbin,zbin,bins_zb,bins_zb_4p,bins_xb,clst_l1,clst_l2,clst_l3
roi,,,,,,,,,,,,,,,,,,,,,
1,21.728455,54.321137,0.000000,0.000000,36.938373,70.255337,5.069973,2.172845,15.934200,132.435176,...,0.000000,"(262.626, 266.667]","(246.465, 250.505]","(157.576, 161.616]","(150, 200]","(100, 200]","(250, 300]",2,7,17
2,3.314009,28.997579,1.657004,1.657004,16.570045,84.507229,3.314009,3.314009,16.570045,24.855067,...,0.000000,"(323.232, 327.273]","(254.545, 258.586]","(294.949, 298.99]","(250, 300]","(200, 300]","(250, 300]",2,7,17
3,6.277141,38.447487,0.000000,6.277141,10.984996,112.203892,3.138570,10.200354,17.262137,9.415711,...,0.000000,"(331.313, 335.354]","(343.434, 347.475]","(157.576, 161.616]","(150, 200]","(100, 200]","(300, 350]",2,9,19
4,23.990102,35.279562,2.116774,0.000000,15.523007,69.853532,3.527956,4.233547,5.644730,261.532646,...,0.000000,"(319.192, 323.232]","(303.03, 307.071]","(278.788, 282.828]","(250, 300]","(200, 300]","(300, 350]",2,7,17
5,6.773756,21.073906,0.752640,1.505279,27.847662,57.200603,3.763198,3.010558,16.558069,8.279035,...,0.000000,"(218.182, 222.222]","(234.343, 238.384]","(278.788, 282.828]","(250, 300]","(200, 300]","(200, 250]",2,7,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9951,4.183755,6.275633,0.000000,0.000000,10.459389,69.031965,0.000000,0.000000,4.183755,106.685764,...,0.000000,"(254.545, 258.586]","(109.091, 113.131]","(339.394, 343.434]","(300, 350]","(300, 400]","(100, 150]",2,8,18
9987,0.000000,11.862459,0.000000,0.000000,5.083911,40.671289,18.641007,11.862459,13.557096,0.000000,...,0.000000,"(177.778, 181.818]","(145.455, 149.495]","(113.131, 117.172]","(100, 150]","(100, 200]","(100, 150]",4,12,25
10009,0.000000,0.000000,6.177442,0.000000,4.118294,4.118294,2.059147,2.059147,0.000000,8.236589,...,0.000000,"(294.949, 298.99]","(101.01, 105.051]","(323.232, 327.273]","(300, 350]","(300, 400]","(100, 150]",3,10,20


In [13]:
len(roi), len(res)

(9723, 8535)

In [14]:
comm = np.intersect1d(roi.index.values, 
                      res.index.values,
                     )
comm.shape

(8535,)

In [15]:
cluster_cols = ['clst_l1', 'clst_l2', 'clst_l3'] 
roi2 = roi.join(res[cluster_cols]).dropna()
roi2[cluster_cols] = roi2[cluster_cols].astype(int)  

roi2sub = roi2[roi2['clst_l1']==1]
_z = roi2sub['z'].values*2/0.84 # convert to pixel
_y = roi2sub['y'].values*2/0.92 # convert to pixel
_x = roi2sub['x'].values*2/0.92 # convert to pixel

conditions = [
    _x <= x_max,
    _y <= y_max,
    _z <= z_max,
    
    _x >  x_min,
    _y >  y_min,
    _z >  z_min,
]
cond = (np.sum(conditions, axis=0) == 6) # all six conditions has to be satified to be pulled
points = np.vstack([_z-z_min,
                    _y-y_min,
                    _x-x_min]).T[cond]

In [16]:
# visualize the centroids of one cell type
viewer.add_points(points, size=10, face_color='blue',edge_color='blue',name='cluster 1')

C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\numpy\core\numeric.py:2449: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


<Points layer 'cluster 1' at 0x23449b249a0>

In [17]:
roi2['clst_l1'].unique()

array([2, 4, 1, 3])

In [18]:
# visualize masks of all cell types (in diff colors)
lookup = roi2['clst_l1']
roiidx, inv = np.unique(lb_sub, return_inverse=True)
# np.all(lb_sub == roiidx[inv].reshape(lb_sub.shape))
clstlbs = lookup.reindex(roiidx).fillna(0).astype(int).values #.values
masks_byclsts = clstlbs[inv].reshape(lb_sub.shape)
viewer.add_labels(masks_byclsts.astype('uint32'), name='clusters',blending='additive')

<Labels layer 'clusters' at 0x2345152aa90>